In [33]:
import json
import requests
import env
import pandas as pd

class Data:
    METABASE_SESSION = ""
def getMetabaseSession(username, password):
    payload = {
        "username": username,
        "password": password
    }
    headers = {
        'content-type': "application/json",
    }
    response = requests.request("POST", env.METABASE_API_GET_SESSION, data=json.dumps(payload, separators=(',', ':')), headers=headers)
    Data.METABASE_SESSION = json.loads(response.text)['id']

def getMetaCards(card_ids):
    responses = []
    for card_id in card_ids:
        url = "http://metabase.jabama.com/api/card/" + str(card_id) + "/query"

        headers = {
            'Content-Type': "application/json",
            'X-Metabase-Session': Data.METABASE_SESSION,
            }

        responses.append([card_id, requests.request("POST", url, headers=headers)])
    return responses

def getDataFromPivotResponses(responses):
    responses_data = []
    
    for response in responses:
        cols = json.loads(response[1].text)['data']['cols']
        rows = json.loads(response[1].text)['data']['rows']
        card_id = response[0]
        col_titles = []
        for col in cols:
            col_titles.append(col['name'])

        responses_data.append({
            "card_id": card_id,
            "columns": col_titles,
            "rows": rows
        })
    return responses_data

def transformPivotResponseToDataFrame(pivotResponse):
    job_audit_df = pd.DataFrame(
        pivotResponse.get("rows"),
        columns = pivotResponse.get("columns")
    )[
        [
            "id",
            "OrderId",
            "PlaceCategory",
            "WorkflowStepId",
            "previous_workflowstepid",
            "next_workflowstepid",
            "nth"
        ]
    ]
    return job_audit_df

def extractPrebookOrders(job_audit_df):
    prebook_orders_list = []
    prebook_flag = 0
    for indx, job_audit in job_audit_df.iterrows():
        
        if job_audit_df["nth"].iloc[indx] == 1:
            if prebook_flag == 1:
                prebook_flag = 0
                prebook_orders_list.append(prebook_list)
            if job_audit_df["WorkflowStepId"].iloc[indx] == "init" and job_audit_df["next_workflowstepid"].iloc[indx] == "init":
                prebook_flag = 1
                prebook_list = []
        if prebook_flag == 1:
            prebook_list.append(job_audit)
    return prebook_orders_list
    
def extractInstantOrders(job_audit_df):
    instant_orders_list = []
    instant_flag = 0
    for indx, job_audit in job_audit_df.iterrows():     
        if job_audit_df["nth"].iloc[indx] == 1:
            if instant_flag == 1:
                instant_flag = 0
                instant_orders_list.append(instant_list)
            if job_audit_df["WorkflowStepId"].iloc[indx] == "init" and job_audit_df["next_workflowstepid"].iloc[indx] != "init":
                instant_flag = 1
                instant_list = []
        if instant_flag == 1:
            instant_list.append(job_audit)
    return instant_orders_list
    

def metabot_job():
    #Get and store the raw data of union of questions from Metabase.
    pivot_metacard_responses = getMetaCards(["1638"])
    pivot_responses_data = getDataFromPivotResponses(pivot_metacard_responses)
    job_audit_df = transformPivotResponseToDataFrame(pivot_responses_data[0])
    
    prebook_job_audit_df = extractPrebookOrders(job_audit_df)
    instant_job_audit_df = extractInstantOrders(job_audit_df)

    splited = splitOrderPathToSubPaths(prebook_job_audit_df)
    
    
def metabase_get_session_job():
    getMetabaseSession(env.METABASE_USERNAME, env.METABASE_PASSWORD)

metabase_get_session_job()
metabot_job()

[[id                         6115078
OrderId                     765449
PlaceCategory                  هتل
WorkflowStepId                init
previous_workflowstepid       None
next_workflowstepid        payment
nth                              1
Name: 40, dtype: object, id                                6115079
OrderId                            765449
PlaceCategory                         هتل
WorkflowStepId                    payment
previous_workflowstepid              init
next_workflowstepid        paymenttimeout
nth                                     2
Name: 41, dtype: object, id                                6115083
OrderId                            765449
PlaceCategory                         هتل
WorkflowStepId             paymenttimeout
previous_workflowstepid           payment
next_workflowstepid        paymenttimeout
nth                                     3
Name: 42, dtype: object, id                                6115316
OrderId                            765449
PlaceC